In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import pickle

def load_data(url):
    df = pd.read_csv(url)
    return df

def preprocess_data(df):
    # Define columns and handle missing values
    columns_thyroid_csv = ['age', 'sex', 'on_thyroxine', 'query_on_thyroxine',
                           'on_antithyroid_medication', 'sick', 'pregnant', 'thyroid_surgery',
                           'I131_treatment', 'query_hypothyroid', 'query_hyperthyroid', 'lithium',
                           'goitre', 'tumor', 'hypopituitary', 'psych', 'TSH_measured', 'TSH',
                           'T3_measured', 'T3', 'TT4_measured', 'TT4', 'T4U_measured', 'T4U',
                           'FTI_measured', 'FTI', 'TBG_measured', 'TBG', 'referral_source',
                           'Class']
    
    df[columns_thyroid_csv] = df[columns_thyroid_csv].replace('?', np.nan)
    df.drop('TBG', inplace=True, axis=1)
    columns_to_drop = ['TT4', 'TSH', 'FTI', 'T4U', 'T3']
    df.drop(columns_to_drop, inplace=True, axis=1)
    
    # Remove rows with missing values in 'age' and 'sex'
    remove_rows = ['age', 'sex']
    for col in remove_rows:
        df = df[df[col].notna()]
    
    # Convert binary categorical columns to numeric
    cols_to_binary = ['on_thyroxine', 'query_on_thyroxine',
                      'on_antithyroid_medication', 'sick', 'pregnant', 'thyroid_surgery',
                      'I131_treatment', 'query_hypothyroid', 'query_hyperthyroid', 'lithium',
                      'goitre', 'tumor', 'hypopituitary', 'psych', 'TSH_measured',
                      'T3_measured', 'TT4_measured', 'T4U_measured', 'FTI_measured',
                      'TBG_measured']
    
    for column in cols_to_binary:
        df[column] = df[column].map({'f': 0, 't': 1})
    
    # Convert 'sex' column
    df['sex'] = df['sex'].map({'F': 0, 'M': 1})
    
    # One-hot encode 'referral_source'
    df = pd.get_dummies(df, columns=['referral_source'])
    
    # Encode target variable 'Class'
    encode = LabelEncoder().fit(df['Class'])
    df['Class'] = encode.transform(df['Class'])
    
    return df, encode


def get_best_params_for_random_forest(train_x, train_y):
    """
    Method Name: get_best_params_for_random_forest
    Description: Get the parameters for Random Forest Algorithm which give the best accuracy.
                 Use Hyperparameter Tuning.
    Output: The model with the best parameters
    On Failure: Raise an Exception
    """
    print('Entered the get_best_params_for_random_forest method')

    try:
        # Initialize with different combinations of parameters
        param_grid = {
            "n_estimators": [50, 100, 150, 200],  # Adjusting number of trees in the forest
            "criterion": ['gini', 'entropy'],
            "max_depth": [None, 10, 20, 30],  # Adjusting maximum depth of each tree
            "max_features": ['auto', 'sqrt', 'log2']  # Adjusting the number of features to consider at every split
        }

        clf = RandomForestClassifier()

        # Create an object of the Grid Search class
        grid = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, verbose=3)

        # Find the best parameters
        grid.fit(train_x, train_y)

        # Extract the best parameters
        best_params = grid.best_params_
        criterion = best_params['criterion']
        max_depth = best_params['max_depth']
        max_features = best_params['max_features']
        n_estimators = best_params['n_estimators']

        # Create a new model with the best parameters
        clf = RandomForestClassifier(n_estimators=n_estimators, criterion=criterion,
                                     max_depth=max_depth, max_features=max_features)

        # Train the new model
        clf.fit(train_x, train_y)

        print('Random Forest best params:', best_params)
        print('Exited the get_best_params_for_random_forest method')

        return clf

    except Exception as e:
        raise Exception("Error occurred during parameter tuning")
    

def train_rf_model(X, y):
    # Oversample to handle class imbalance
    ros = RandomOverSampler(random_state=2)
    X_sampled, y_sampled = ros.fit_resample(X, y)
    
    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X_sampled, y_sampled, test_size=0.3, random_state=2)

    rfc_model = get_best_params_for_random_forest(X_train, y_train)
    
    # Make predictions
    rfc_pred = rfc_model.predict(X_test)
    
    # Print classification report
    print(classification_report(y_test, rfc_pred))
    
    return rfc_model

def save_model(model, encoder, model_filename, encoder_filename):
    # Save the model to disk using pickle
    with open(model_filename, 'wb') as model_file:
        pickle.dump(model, model_file)
    
    # Save the encoder to disk using pickle
    with open(encoder_filename, 'wb') as encoder_file:
        pickle.dump(encoder, encoder_file)

In [2]:

path = 'hypothyroid.csv'
df = load_data(path)
df_preprocessed, encoder = preprocess_data(df)
X = df_preprocessed.drop('Class', axis=1)
y = df_preprocessed['Class']
trained_model = train_rf_model(X, y)


model_filename = "model/trained_model.pkl"
encoder_filename = "model/label_encoder.pkl"

# Save the trained model and encoder
save_model(trained_model, encoder, model_filename, encoder_filename)

Entered the get_best_params_for_random_forest method
Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV 1/5] END criterion=gini, max_depth=None, max_features=auto, n_estimators=50;, score=0.900 total time=   0.0s


c:\Users\priks\anaconda3\envs\side\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
c:\Users\priks\anaconda3\envs\side\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV 2/5] END criterion=gini, max_depth=None, max_features=auto, n_estimators=50;, score=0.890 total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=None, max_features=auto, n_estimators=50;, score=0.908 total time=   0.0s


c:\Users\priks\anaconda3\envs\side\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
c:\Users\priks\anaconda3\envs\side\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV 4/5] END criterion=gini, max_depth=None, max_features=auto, n_estimators=50;, score=0.908 total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=None, max_features=auto, n_estimators=50;, score=0.881 total time=   0.0s


...

[CV 4/5] END criterion=entropy, max_depth=30, max_features=auto, n_estimators=200;, score=0.909 total time=   0.5s


c:\Users\priks\anaconda3\envs\side\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV 5/5] END criterion=entropy, max_depth=30, max_features=auto, n_estimators=200;, score=0.879 total time=   0.5s
[CV 1/5] END criterion=entropy, max_depth=30, max_features=sqrt, n_estimators=50;, score=0.897 total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=30, max_features=sqrt, n_estimators=50;, score=0.891 total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=30, max_features=sqrt, n_estimators=50;, score=0.908 total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=30, max_features=sqrt, n_estimators=50;, score=0.909 total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=30, max_features=sqrt, n_estimators=50;, score=0.880 total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=30, max_features=sqrt, n_estimators=100;, score=0.899 total time=   0.2s
[CV 2/5] END criterion=entropy, max_depth=30, max_features=sqrt, n_estimators=100;, score=0.891 total time=   0.2s
[CV 3/5] END criterion=entropy, max_depth=30, max_features=sqrt, n_estimators=100;, s

c:\Users\priks\anaconda3\envs\side\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
c:\Users\priks\anaconda3\envs\side\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Random Forest best params: {'criterion': 'entropy', 'max_depth': None, 'max_features': 'auto', 'n_estimators': 100}
Exited the get_best_params_for_random_forest method
              precision    recall  f1-score   support

           0       0.81      0.86      0.84       996
           1       0.98      0.77      0.86      1032
           2       0.81      0.95      0.87       977
           3       1.00      1.00      1.00      1005

    accuracy                           0.89      4010
   macro avg       0.90      0.89      0.89      4010
weighted avg       0.90      0.89      0.89      4010

